In [2]:
#Necesary imports.
import bitso
import pandas as pd
import numpy as np
from datetime import timezone
import datetime
from dateutil.tz import tzutc
import time
from fbprophet import Prophet
import matplotlib.pyplot as plt
from scipy.stats import boxcox
import warnings
warnings.filterwarnings('ignore')
import plotly.offline
from plotly import graph_objs as go

In [3]:
#Opening my api key and secret.
with open('../key.txt') as f:
    a = f.readlines()

with open('../secret.txt') as p:
    b = p.readlines()

#Using the api secret and key to call the api.
API_SECRET = [i.split('\n') for i in a][0][0]
API_KEY = [i.split('\n') for i in b][0][0]
api = bitso.Api(API_KEY, API_SECRET)

In [4]:
'''
marker for last_scrap on 14-1-20: 13186700, I save this marker so i can use it
the next time i'm scrapping
'''
#Defining the empyt lists of the elements we need from the api.
btc_mxn_date = []
btc_mxn_price = []
btc_mxn_volume = []
btc_mxn_hour = []
btc_mxn_side = []

#Marker than changes the time in the api calls.
mark = 13186700

#Datetime when the code is running
now = datetime.datetime.now(timezone.utc)

#While loop for scrapping the information we need
while 1:
    #limit of requests, the max we can get is 100
    limit = '100'
    #Calling the api trades, we use sort 'asc' so i't starts on the marker we want.
    trades = api.trades('btc_mxn', sort='asc', limit=limit, marker=mark)
    #Starting the counter so we don't raise the error of too many requests on the api.
    count = 0
    #printing the mark and the spaces, we need these so we know the last mark we use.
    print(mark)
    print('')
    print('')
    #if statement to break the while loop.
    if trades != []:
        #starting a for loop to append the elements we need from each trade to our lists
        for e in trades:
            #adding to the counter.
            count +=1
            # if statement so we just scrap elements before today.
            if e.created_at < now :
                #printing each trade so we can see that everythings is working fine.
                print(e)
                #scrapping the elements we need
                btc_mxn_price.append(e.price)
                btc_mxn_volume.append(e.amount)
                btc_mxn_hour.append(e.created_at.hour)
                btc_mxn_date.append(e.created_at.date())
                btc_mxn_side.append(e.maker_side)
        '''
        if statement that starts a timer when the count reachs 100, we can just get
        300 requests per minute so our timer waits 21 seconds before making the next
        100 requests. It also adds a thousand to our mark so it moves forward in time.
        '''
        if count == 100:
            time.sleep(21)
            count = 0
            mark = mark + 1000
            
    #else statement to break the loop.
    else:
        break
        

13186700


Trade(tid=13186706, price=162390.05, amount=0.00011084, maker_side=sell, created_at=2020-01-14 17:00:15+00:00)
Trade(tid=13186755, price=162000.00, amount=0.02321700, maker_side=sell, created_at=2020-01-14 17:00:56+00:00)
Trade(tid=13186759, price=162000.00, amount=0.01538800, maker_side=sell, created_at=2020-01-14 17:00:57+00:00)
Trade(tid=13186761, price=162000.00, amount=0.02174300, maker_side=sell, created_at=2020-01-14 17:01:01+00:00)
Trade(tid=13186762, price=162000.00, amount=0.02174300, maker_side=sell, created_at=2020-01-14 17:01:02+00:00)
Trade(tid=13186763, price=162000.00, amount=0.01290400, maker_side=sell, created_at=2020-01-14 17:01:03+00:00)
Trade(tid=13186764, price=162000.00, amount=0.01290400, maker_side=sell, created_at=2020-01-14 17:01:03+00:00)
Trade(tid=13186765, price=162000.00, amount=0.02321600, maker_side=sell, created_at=2020-01-14 17:01:04+00:00)
Trade(tid=13186766, price=162000.00, amount=0.01394300, maker_side=sell, created_at=2020-01-14 17:01:

13187700


Trade(tid=13187707, price=163201.08, amount=0.03443273, maker_side=buy, created_at=2020-01-14 17:21:31+00:00)
Trade(tid=13187709, price=164154.73, amount=0.00609181, maker_side=sell, created_at=2020-01-14 17:21:41+00:00)
Trade(tid=13187713, price=163201.12, amount=0.00308375, maker_side=buy, created_at=2020-01-14 17:21:52+00:00)
Trade(tid=13187722, price=164153.94, amount=0.00609184, maker_side=sell, created_at=2020-01-14 17:22:42+00:00)
Trade(tid=13187730, price=164104.50, amount=0.02104305, maker_side=sell, created_at=2020-01-14 17:23:13+00:00)
Trade(tid=13187732, price=163700.00, amount=0.00188296, maker_side=buy, created_at=2020-01-14 17:23:18+00:00)
Trade(tid=13187733, price=163500.01, amount=0.02889586, maker_side=buy, created_at=2020-01-14 17:23:18+00:00)
Trade(tid=13187737, price=163500.01, amount=0.03750386, maker_side=buy, created_at=2020-01-14 17:23:40+00:00)
Trade(tid=13187740, price=163870.20, amount=0.00021102, maker_side=sell, created_at=2020-01-14 17:23:51+00

13188700


Trade(tid=13188712, price=164743.24, amount=0.00218600, maker_side=sell, created_at=2020-01-14 17:47:26+00:00)
Trade(tid=13188726, price=164743.23, amount=0.00607005, maker_side=sell, created_at=2020-01-14 17:47:45+00:00)
Trade(tid=13188732, price=164523.20, amount=0.05000000, maker_side=buy, created_at=2020-01-14 17:47:56+00:00)
Trade(tid=13188739, price=164743.24, amount=0.03035026, maker_side=sell, created_at=2020-01-14 17:48:07+00:00)
Trade(tid=13188753, price=164743.24, amount=0.00971208, maker_side=sell, created_at=2020-01-14 17:48:17+00:00)
Trade(tid=13188754, price=164743.24, amount=0.06588392, maker_side=sell, created_at=2020-01-14 17:48:17+00:00)
Trade(tid=13188755, price=164743.25, amount=0.00079302, maker_side=sell, created_at=2020-01-14 17:48:17+00:00)
Trade(tid=13188756, price=164775.00, amount=0.00045517, maker_side=sell, created_at=2020-01-14 17:48:17+00:00)
Trade(tid=13188760, price=164800.00, amount=0.00022185, maker_side=sell, created_at=2020-01-14 17:48:2

13189700


Trade(tid=13189701, price=165212.00, amount=0.02667572, maker_side=buy, created_at=2020-01-14 18:10:23+00:00)
Trade(tid=13189711, price=165979.84, amount=0.00030124, maker_side=sell, created_at=2020-01-14 18:10:37+00:00)
Trade(tid=13189712, price=165978.53, amount=0.00171526, maker_side=sell, created_at=2020-01-14 18:10:39+00:00)
Trade(tid=13189713, price=165978.53, amount=0.00168202, maker_side=sell, created_at=2020-01-14 18:10:39+00:00)
Trade(tid=13189714, price=165978.54, amount=0.05082660, maker_side=sell, created_at=2020-01-14 18:10:39+00:00)
Trade(tid=13189716, price=165212.11, amount=0.00312421, maker_side=buy, created_at=2020-01-14 18:10:47+00:00)
Trade(tid=13189721, price=165212.18, amount=0.01218485, maker_side=buy, created_at=2020-01-14 18:11:11+00:00)
Trade(tid=13189727, price=165999.97, amount=0.00301205, maker_side=sell, created_at=2020-01-14 18:11:36+00:00)
Trade(tid=13189737, price=165999.97, amount=0.00354753, maker_side=sell, created_at=2020-01-14 18:11:52+

13190700


Trade(tid=13190701, price=166124.98, amount=0.00339955, maker_side=sell, created_at=2020-01-14 18:31:39+00:00)
Trade(tid=13190742, price=165800.00, amount=0.00015178, maker_side=buy, created_at=2020-01-14 18:32:46+00:00)
Trade(tid=13190793, price=165800.00, amount=0.00182125, maker_side=buy, created_at=2020-01-14 18:33:10+00:00)
Trade(tid=13190808, price=165800.00, amount=0.01572507, maker_side=buy, created_at=2020-01-14 18:33:13+00:00)
Trade(tid=13190820, price=165799.96, amount=0.00004250, maker_side=buy, created_at=2020-01-14 18:33:16+00:00)
Trade(tid=13190864, price=165799.96, amount=0.00396253, maker_side=buy, created_at=2020-01-14 18:33:25+00:00)
Trade(tid=13190888, price=165156.02, amount=0.00980000, maker_side=buy, created_at=2020-01-14 18:33:32+00:00)
Trade(tid=13190900, price=165375.89, amount=0.00292688, maker_side=sell, created_at=2020-01-14 18:33:43+00:00)
Trade(tid=13190901, price=165156.00, amount=0.03611011, maker_side=buy, created_at=2020-01-14 18:33:45+00:0

...

13301700


Trade(tid=13301706, price=161787.32, amount=0.00019900, maker_side=buy, created_at=2020-01-22 22:57:30+00:00)
Trade(tid=13301708, price=162284.32, amount=0.00030810, maker_side=sell, created_at=2020-01-22 22:57:52+00:00)
Trade(tid=13301709, price=162284.31, amount=0.01540506, maker_side=sell, created_at=2020-01-22 22:57:55+00:00)
Trade(tid=13301724, price=161787.32, amount=0.00465361, maker_side=buy, created_at=2020-01-22 22:58:43+00:00)
Trade(tid=13301726, price=161787.32, amount=0.00441719, maker_side=buy, created_at=2020-01-22 22:58:52+00:00)
Trade(tid=13301727, price=162279.52, amount=0.15037176, maker_side=sell, created_at=2020-01-22 22:59:16+00:00)
Trade(tid=13301728, price=162279.19, amount=0.00750000, maker_side=sell, created_at=2020-01-22 22:59:48+00:00)
Trade(tid=13301734, price=161787.32, amount=0.09523868, maker_side=buy, created_at=2020-01-22 23:00:02+00:00)
Trade(tid=13301737, price=162278.61, amount=0.07394690, maker_side=sell, created_at=2020-01-22 23:00:21+0

13302700


Trade(tid=13302702, price=161299.00, amount=0.00096519, maker_side=buy, created_at=2020-01-23 01:05:01+00:00)
Trade(tid=13302703, price=161261.70, amount=0.00372067, maker_side=buy, created_at=2020-01-23 01:05:01+00:00)
Trade(tid=13302704, price=161219.43, amount=0.00595514, maker_side=buy, created_at=2020-01-23 01:05:01+00:00)
Trade(tid=13302708, price=161326.57, amount=0.00323774, maker_side=sell, created_at=2020-01-23 01:05:32+00:00)
Trade(tid=13302709, price=161326.58, amount=0.02465599, maker_side=sell, created_at=2020-01-23 01:05:32+00:00)
Trade(tid=13302710, price=161212.11, amount=0.01100000, maker_side=buy, created_at=2020-01-23 01:05:35+00:00)
Trade(tid=13302711, price=161326.58, amount=0.16493779, maker_side=sell, created_at=2020-01-23 01:05:37+00:00)
Trade(tid=13302712, price=161400.31, amount=0.24635401, maker_side=sell, created_at=2020-01-23 01:05:37+00:00)
Trade(tid=13302713, price=161739.83, amount=0.04817996, maker_side=sell, created_at=2020-01-23 01:05:37+0

13303700


Trade(tid=13303702, price=160435.71, amount=0.00060523, maker_side=sell, created_at=2020-01-23 02:21:55+00:00)
Trade(tid=13303704, price=160350.00, amount=0.00069049, maker_side=buy, created_at=2020-01-23 02:22:07+00:00)
Trade(tid=13303705, price=160350.00, amount=0.05292489, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303706, price=160323.05, amount=0.24814365, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303707, price=160290.25, amount=0.00058362, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303708, price=160286.00, amount=0.00187000, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303709, price=160160.00, amount=0.35782031, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303710, price=160158.37, amount=0.00093658, maker_side=buy, created_at=2020-01-23 02:22:08+00:00)
Trade(tid=13303711, price=160100.20, amount=0.02498436, maker_side=buy, created_at=2020-01-23 02:22:08+00:00

13304700


Trade(tid=13304705, price=160597.18, amount=0.00030858, maker_side=sell, created_at=2020-01-23 04:57:39+00:00)
Trade(tid=13304706, price=160597.18, amount=0.02802042, maker_side=sell, created_at=2020-01-23 04:58:23+00:00)
Trade(tid=13304707, price=160508.98, amount=0.02500000, maker_side=buy, created_at=2020-01-23 04:59:11+00:00)
Trade(tid=13304708, price=160508.98, amount=0.00630000, maker_side=buy, created_at=2020-01-23 04:59:19+00:00)
Trade(tid=13304709, price=160597.18, amount=0.00062268, maker_side=sell, created_at=2020-01-23 04:59:24+00:00)
Trade(tid=13304710, price=160508.99, amount=0.00042194, maker_side=buy, created_at=2020-01-23 04:59:57+00:00)
Trade(tid=13304714, price=160597.16, amount=0.01245352, maker_side=sell, created_at=2020-01-23 05:01:31+00:00)
Trade(tid=13304720, price=160508.97, amount=0.01092149, maker_side=buy, created_at=2020-01-23 05:02:04+00:00)
Trade(tid=13304723, price=160500.00, amount=0.06230530, maker_side=buy, created_at=2020-01-23 05:02:25+00

13305700




In [7]:
# creating a new data frame.
df = pd.DataFrame(columns=['price', 'volume', 'hour', 'date', 'side'])
# using the lists we got from the while loop to fill the dataframe.
df.price = btc_mxn_price
df.volume = btc_mxn_volume
df.hour = btc_mxn_hour
df.date = btc_mxn_date
df.side = btc_mxn_side
df.date = df.date.astype('datetime64')
#creating new columns with the month and day of each row.
df['month'] = [i.month for i in df.date]
df['day'] = [i.day for i in df.date]

In [8]:
#Creating a function that needs a data frame and a number(month) for input.
def extraction(df, m):
    #variable that get's just the row that matches the same month
    f_df = df.loc[df['month'] == m]
    #variable that stores the year
    y = datetime.datetime.today().year
    #variable that creates a list with the numbers of days of the month
    days = list(f_df['day'].unique())
    #varible that creates a empty dictionary
    results = {}
    #list with the names of the keys in the dictionary that the function returns.
    nombres = ['date', '24h_avg', '12h_avg', 'min', 'max', 'volume', 'm_side']
    
    #for loop that creates new keys in the empty dictionary with a empty list as values.
    for i in nombres:
        results[i] = []
    
    #for that loops on each day in the list of the days of the month.
    for e in days:
        #variable that get's just the rows that match the day in the for loop.
        ndf = f_df.loc[f_df.day == e]
        #appending the results in each dictionary key.
        results['date'].append(datetime.date(y, m, e))
        results['24h_avg'].append(ndf.price.mean())
        results['12h_avg'].append(ndf.price.loc[ndf.hour<12].mean())
        results['max'].append(ndf.price.max())
        results['min'].append(ndf.price.min())
        results['volume'].append(ndf.volume.sum())
        weight = ndf.side.value_counts().sum()
        buy = ndf.side.value_counts()['buy']/weight
        sell = ndf.side.value_counts()['sell']/weight
        if buy>sell:
            results['m_side'].append('buy')
        else:
            results['m_side'].append('sell')
            
    #returns the dictionary with the appended elements
    return results

In [9]:
#creating a variable that stores the month we are in.
month = datetime.datetime.today().month
#creating a new data frame using the dictionary from the above function and the month
df_new = pd.DataFrame.from_dict(extraction(df, month))

In [19]:
#Reading our csv file where we store the past data
history = pd.read_csv('../csv_files/historic_data.csv')

In [34]:
btc_df = pd.concat([history, df_new], ignore_index=True)

In [36]:
btc_df.drop_duplicates(inplace=True)

In [37]:
btc_df.drop(btc_df.tail(1).index,inplace=True)

In [38]:
btc_df.to_csv('../csv_files/historic_data.csv', index=False)

In [41]:
btc_df.date = btc_df.date.astype('datetime64')
btc_df['min'] = btc_df['min'].astype('float')
btc_df['max'] = btc_df['max'].astype('float')
btc_df.volume = btc_df.volume.astype('float')
btc_df['m_side'] = btc_df['m_side'].astype('category')

In [42]:
df_dum = pd.get_dummies(btc_df)
df_dum.rename(columns={'m_side_buy':'buy', 'm_side_sell':'sell'}, inplace=True)

In [43]:
change = []
for i in range(1,len(df_dum)):
    if df_dum.sell.iloc[i] != df_dum.sell.iloc[i-1]:
        change.append(df_dum.date.iloc[i])

In [46]:
#Selecting the columns we need for fbprophet
btc_24h_avg = btc_df[['date', '24h_avg']].copy()
#Changing the columns names for fbprophet to accept it.
btc_24h_avg.rename(columns={'date': 'ds', '24h_avg': 'y'}, inplace=True)

In [47]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

In [48]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [49]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    df.dropna(inplace=True)
    
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [50]:
train_df = btc_24h_avg.copy().set_index('ds')

In [51]:
train_df['y'], lambda_prophet = boxcox(train_df['y'])
train_df.reset_index(inplace=True)

In [52]:
m2 = Prophet(changepoints=change, seasonality_mode='multiplicative')
m2.fit(train_df)
future2 = m2.make_future_dataframe(periods=15)
forecast2 = m2.predict(future2)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [53]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column],
                                       lambda_prophet)

In [54]:
cmp_df2 = make_comparison_dataframe(btc_24h_avg, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, 15).items():
     print(err_name, err_value)

MAPE 8.520746861603978
MAE 13859.720616855153


In [57]:


def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Zona de compra', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="green"))
    upper_bound = create_go('Máximo esperado', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="red"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Venta sugerida', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Precio promedio del día', 'y', num_values,
                       marker=dict(color="red"))
    
    
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='MXN'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.plot(fig, filename='btc_mxn.html', auto_open=False)

show_forecast(cmp_df2, 15, 2123, 'BTC')

